In [1]:
import gym
from gym import error
from gym.utils import closer
from PIL import Image
import torch
from pypylon import pylon
import numpy as np
import cv2
from ctypes import *
import glob
import matplotlib.pylab as pl
import matplotlib.pyplot as plt
import gym
from gym import error
from gym.utils import closer
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common import make_vec_env
from stable_baselines import A2C, PPO2
from gym import spaces
from pypylon import pylon
import stable_baselines
import time
from matplotlib import cm
from stable_baselines.common.schedules import LinearSchedule
import random
%matplotlib inline

env_closer = closer.Closer()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
class CorningEnv(gym.Env):
    """The main OpenAI Gym class. It encapsulates an environment with
    arbitrary behind-the-scenes dynamics. An environment can be
    partially or fully observed.
    The main API methods that users of this class need to know are:
        step
        reset
        render
        close
        seed
    And set the following attributes:
        action_space: The Space object corresponding to valid actions
        observation_space: The Space object corresponding to valid observations
        reward_range: A tuple corresponding to the min and max possible rewards
    Note: a default reward range set to [-inf,+inf] already exists. Set it if you want a narrower range.
    The methods are accessed publicly as "step", "reset", etc...
    
    """
    metadata = {'render.modes': ['rgb_array']}
    
    def __init__(self, camera,
                     Width = 417,
                     Height = 404,
                     threshold = 85,
                     file_path = r'C:\Users\CIG\Documents\corning_basler_RL\new_obs'):
        
        
        # Create an instant camera object with the camera device found first.
        #self.camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
        #self.camera.Open()
        self.camera = camera
        self.camera.AcquisitionFrameRateAbs.Value = 10
        self.camera.GainRaw.Value = 36
        self.camera.AcquisitionMode.SetValue('Continuous')
        self.Width = Width
        self.Height = Height
        self.camera.Width.Value = Width
        self.camera.Height.Value = Height
        print(self.camera.Width.GetValue(), self.camera.Height.GetValue())
        # code from https://github.com/basler/pypylon/blob/master/samples/opencv.py
        img = pylon.PylonImage()
        self.converter = pylon.ImageFormatConverter()
        # converting to opencv bgr format
        self.converter.OutputPixelFormat = pylon.PixelType_BGR8packed
        self.converter.OutputBitAlignment = pylon.OutputBitAlignment_MsbAligned
        
        
        # corning lib
        self.lib = cdll.LoadLibrary(r"C:\Users\CIG\Documents\MATLAB\ComCasp64.dll")
        #Check if Maxim driver dll is loaded
        eCOMCaspErr = getattr(self.lib,'Casp_OpenCOM')
        print('eCOMCaspErr:', eCOMCaspErr(), self.lib.Casp_OpenCOM())
        
        
        # yolo
        self.model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5s', 
                                         force_reload=True, 
                                         pretrained=True)
        
        
        
        self.range_ex_time = np.concatenate((np.arange(1e3, 45e3, 1000), 
                                             np.arange(50e3, 500e3, 5000), 
                                             np.arange(600e3, 900e3, 50000), 
                                             np.arange(100e4, 500e4, 500000))).astype(np.int64)
        
        number_of_actions = len(self.range_ex_time)
        
        # environment parameters 
        # coarse V
        # fine V
        # 
        self.action_space = spaces.MultiDiscrete([69, 99, number_of_actions])
        
        # image
        self.observation_space = spaces.Box(low=0, 
                                            high=255, 
                                            shape=(2, Height, Width, 3),
                                            dtype = np.uint8)
        
        
        self.threshold = threshold
        self.file_path = file_path
        self.i = 0
        self.i2 = 0
        self.done_camera = False
        self.observation_from_camera_buffer = np.zeros((self.Height, self.Width, 3))
        time.sleep(1)
        
    def grab_image(self):
        # https://github.com/basler/pypylon/blob/master/samples/opencv.py
        self.camera.StartGrabbing()
        while 1:
            grabResult = self.camera.RetrieveResult(5000, pylon.TimeoutHandling_ThrowException)
            if grabResult.GrabSucceeded():
                # Access the image data
                image = self.converter.Convert(grabResult)
                image = image.GetArray()
                #print(img[0])
                #print('shape:', img.shape)
                #plt.imshow(img)
                break
        self.camera.StopGrabbing()
        return image
        
    def reset(self):
        """Resets the environment to an initial state and returns an initial
        observation.
        Note that this function should not reset the environment's random
        number generator(s); random variables in the environment's state should
        be sampled independently between multiple calls to `reset()`. In other
        words, each call of `reset()` should yield an environment suitable for
        a new episode, independent of previous episodes.
        Returns:
            observation (object): the initial observation.
        """
        self.done_camera = False
        
        # https://docs.baslerweb.com/features
        self.camera.ExposureTimeRaw.Value = 20000
        
        self.lens_movement(24.0)
        
        self.i = 0
        self.i2 = 0
        time.sleep(1)
        
        obs = self.grab_image().astype(np.uint8)
        
        self.observation_from_camera_buffer = obs
        
        observation = np.stack((obs, obs))
        
        return observation
        
    def step(self, action):
        """Run one timestep of the environment's dynamics. When end of
        episode is reached, you are responsible for calling `reset()`
        to reset this environment's state.
        Accepts an action and returns a tuple (observation, reward, done, info).
        Args:
            action (object): an action provided by the agent
        Returns:
            observation (object): agent's observation of the current environment
            reward (float) : amount of reward returned after previous action
            done (bool): whether the episode has ended, in which case further step() calls will return undefined results
            info (dict): contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        print(action)
        if self.done_camera == False:
            action_camera = action[2]
            obs_camera, reward_camera, done_camera_ = self.camera_env(action_camera)
            self.done_camera = done_camera_
            print('Done camera:', self.done_camera)
            if self.done_camera == False:
                self.i2+=1
                observation = np.stack((np.zeros((self.Height, self.Width, 3)), obs_camera))
                return observation, reward_camera, self.done_camera, {}
            else:
                #print('TRUE')
                self.observation_from_camera_buffer = obs_camera
                obs_lens, reward, done = self.lens_env(action)
                self.i+=1 
                observation = np.stack((obs_lens, obs_camera))
                
        else:
            #print('TRUE2')
            obs_lens, reward, done = self.lens_env(action)
            self.i+=1 
            observation = np.stack((obs_lens, self.observation_from_camera_buffer))
            if self.i%5 == 0 and done == False:
                self.done_camera = False
                
        return observation, reward, done, {}
    
    def lens_env(self, action):
        
        action1 = 0
        action2 = action[1]
        if action[0] < 24:
            action1 = 24
        else:
            action1 = action[0]
        action = float("%s.%s"% (str(action1), str(action2)))

        self.lens_movement(action)
 
        camera_observation = self.grab_image()
        prediction_yolo = self.model_yolo(camera_observation)
        final_result_yolo = self.find_class(prediction_yolo)
        
        if final_result_yolo.size == 0:
            print('not detected')
            foc_value = -1
            betta = 0.5
            reward = -100-betta*self.i
            #reward = -100
            
        else:
            image = Image.fromarray(camera_observation)
            image = image.crop((final_result_yolo[0], 
                             final_result_yolo[1], 
                             final_result_yolo[2], 
                             final_result_yolo[3]))
            foc_value = self.focus_value(image)
            
        if foc_value > self.threshold: #or self.i == 20:
            done = True
            reward = 1
            print('------',True, action, self.i, '------')
        else:
            done = False
            reward = 1 - 0.001*(foc_value - self.threshold)**2
            #reward = -1
            print(False, action)
            
        im = Image.fromarray(camera_observation)
        filename = self.file_path + "\img_%s_%s.png" % (str(action), str(done))
        im.save(filename)

        #print(camera_observation)
        camera_observation = camera_observation.astype(np.uint8)
            
        return camera_observation, reward, done
    
    def find_class(self, results, obj = 'car'):
        len_of_class = len(results.pred)
        pred_class = [int(results.pred[0][i].numpy()[5]) for i in range(len(results.pred[0]))]
        #list_of_yolo_classes = list(zip(results.names, range(0, len(results.names))))
        pred_names = [results.names[i] for i in pred_class]
        if obj in pred_names:
            index = pred_names.index(obj)
            return results.pred[0][index].numpy()
        else:
            return np.array([])
        
    def focus_value(self, im):
        # Calculate the gradient
        sobelx = cv2.Sobel(np.float32(im), cv2.CV_64F ,1 , 0, ksize=5)
        sobely = cv2.Sobel(np.float32(im),cv2.CV_64F,0,1,ksize=5)

        abs_sobel_x = cv2.convertScaleAbs(sobelx) # converting back to uint8
        abs_sobel_y = cv2.convertScaleAbs(sobely)
        #print(abs_sobel_x )

        # Combine the two gradients with equal weight
        dst = cv2.addWeighted(abs_sobel_x,0.5,abs_sobel_y,0.5,0)
        #print(dst)

        # Calculate the average gradient for the image
        # I convert it to a numpy array for ease of calculation
        return pl.asarray(dst).mean()
    
    def lens_movement(self, action):
        x = c_double(action)
        self.lib.Casp_SetFocusVoltage(x)
        time.sleep(2)
    
    def camera_env(self, action_camera):
        
        ExposureTimeRaw = int(self.range_ex_time[action_camera])
        
#         if action_camera == 0:
#             ExposureTimeRaw = random.randint(1e3,5e3)
#         if action_camera == 1:
#             ExposureTimeRaw = random.randint(6e3,9e3)
#         if action_camera == 2:
#             ExposureTimeRaw = random.randint(10e3,25e3)
#         if action_camera == 3:
#             ExposureTimeRaw = random.randint(26e3,50e3)
#         if action_camera == 4:
#             ExposureTimeRaw = random.randint(51e3,75e3)
#         if action_camera == 5:
#             ExposureTimeRaw = random.randint(76e3,99e3)
#         if action_camera == 6:
#             ExposureTimeRaw = random.randint(100e3,250e3)
#         if action_camera == 7:
#             ExposureTimeRaw = random.randint(260e3,500e3)
#         if action_camera == 8:
#             ExposureTimeRaw = random.randint(510e3,700e3)
#         if action_camera == 9:
#             ExposureTimeRaw = random.randint(710e3,999e3)
#         if action_camera == 10:
#             ExposureTimeRaw = random.randint(100e4,250e4)
#         if action_camera == 11:
#             ExposureTimeRaw = random.randint(260e4,500e4)
#         if action_camera == 12:
#             ExposureTimeRaw = random.randint(510e4,750e4)
#         if action_camera == 13:
#             ExposureTimeRaw = random.randint(760e4,900e4)
            
        print('ExposureTimeRaw:', ExposureTimeRaw)
        self.lens_movement(46.0)
        self.camera.ExposureTimeRaw.Value = ExposureTimeRaw
        time.sleep(5)
        obs_camera = self.grab_image().astype(np.uint8)
        
        hist , bin_edges = np.histogram(np.array(obs_camera).ravel())
        #reward = np.sum(bin_edges) - 250
        max_value_bin = bin_edges[np.argmax(hist)]
        if max_value_bin > 50 and max_value_bin < 170:
            reward_camera = 1
        else:
            reward_camera = -100 - 0.5*self.i2
        
        #hist, _ = np.histogram(np.array(obs_camera).ravel(), 255,[0,255])
        
        #array = np.concatenate((hist, bin_edges))
        #array_null = np.zeros((len(np.array(obs_camera).ravel()) - len(array), ))
        #obs_camera = np.concatenate((array, array_null)).reshape((self.Height, self.Width, 3))
        
        if reward_camera == 1:
            done_camera = True
        else:
            done_camera = False
            
        im = Image.fromarray(obs_camera)
        filename = self.file_path + "\img_ExTime_%s_%s.png" % (str(ExposureTimeRaw), str(done_camera))
        im.save(filename)
            
        return obs_camera, reward_camera, done_camera

In [3]:
# Create an instant camera object with the camera device found first.
camera = pylon.InstantCamera(pylon.TlFactory.GetInstance().CreateFirstDevice())
camera.Open()

In [4]:
env = CorningEnv(camera)

417 404
eCOMCaspErr: 0 0


Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\CIG/.cache\torch\hub\master.zip

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Focus                     [3, 32, 3]                    
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    156928  models.common.C3                        [128, 128, 3]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1    625152  models.common.C3                        [256, 256, 3]                 
  7                -1  1   

Adding autoShape... 


In [5]:
# from stable_baselines.common.env_checker import check_env
# # It will check your custom environment and output additional warnings if needed
# check_env(env)
#env.action_space.low

In [6]:
from stable_baselines.common.vec_env import DummyVecEnv
model = PPO2(MlpPolicy, env, 
             #learning_rate = sched_LR.value,
             verbose=1, 
             tensorboard_log = '')

# Load the trained agent
# model = PPO2.load("model_corning_basler_final_version_mar03", 
#                   env=DummyVecEnv([lambda: env]), 
#                   tensorboard_log = '')

Wrapping the env in a DummyVecEnv.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\tf_util.py:191: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\tf_util.py:200: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\policies.py:116: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\input.py:25: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From C:\Users\CIG\AppData\Roaming\Python\Python37\site-packages\tensorflow_core\python\ops\math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:206: The name tf.train.AdamOptimizer is deprecated. Please use tf.compat.v1.train.AdamOptimizer instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:240: The name tf.global_variables_initializer is deprecated. Please use tf.compat.v1.global_variables_initializer instead.



From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\ppo2\ppo2.py:242: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.



In [7]:
# Enjoy trained agent
# obs = env.reset()
# while True:
#     action, _states = model.predict(obs)
#     obs, rewards, dones, info = env.step(action)

In [ ]:
model.learn(total_timesteps=5000, tb_log_name="corning_basler_new_obs_03", reset_num_timesteps=False)

From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\base_class.py:1169: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



[30 38 50]
ExposureTimeRaw: 80000
Done camera: False
[ 43  27 123]
ExposureTimeRaw: 445000
Done camera: True
not detected
False 43.27
[60 34 40]
not detected
False 60.34
[13 23 96]
not detected
False 24.23
[18  7 95]
not detected
False 24.7
[ 6 48 96]
not detected
False 24.48
[12 70 54]
ExposureTimeRaw: 100000
Done camera: False
[ 46  97 102]
ExposureTimeRaw: 340000
Done camera: True
not detected
False 46.97
[ 35  25 107]
not detected
False 35.25
[ 6 80 30]
not detected
False 24.8
[23  3 20]
not detected
False 24.3
[ 35  61 142]
not detected
False 35.61
[ 3 18 54]
ExposureTimeRaw: 100000
Done camera: False
[  5  77 104]
ExposureTimeRaw: 350000
Done camera: True
not detected
False 24.77
[20 15 44]
not detected
False 24.15
[57 94 72]
not detected
False 57.94
[ 36  58 143]
not detected
False 36.58
[ 11  95 112]
not detected
False 24.95
[ 30  40 103]
ExposureTimeRaw: 345000
Done camera: True
not detected
False 30.4
[ 20  53 138]
not detected
False 24.53
[54 74 80]
not detected
False 54.74


From C:\Users\CIG\Anaconda3\envs\corning\lib\site-packages\stable_baselines\common\tf_util.py:502: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



--------------------------------------
| approxkl           | 0.00044445752 |
| clipfrac           | 0.0           |
| explained_variance | -0.00186      |
| fps                | 0             |
| n_updates          | 1             |
| policy_entropy     | 13.826269     |
| policy_loss        | -0.014858805  |
| serial_timesteps   | 128           |
| time_elapsed       | 0             |
| total_timesteps    | 128           |
| value_loss         | 79988.98      |
--------------------------------------
[25 12 51]
ExposureTimeRaw: 85000
Done camera: False
[ 11  46 121]
ExposureTimeRaw: 435000
Done camera: True
not detected
False 24.46
[26 78 46]
not detected
False 26.78
[44 14 74]
not detected
False 44.14
[65 36  7]
not detected
False 65.36
[25 44 21]
not detected
False 25.44
[24  7 20]
ExposureTimeRaw: 21000
Done camera: False
[20 72 66]
ExposureTimeRaw: 160000
Done camera: False
[35 73 12]
ExposureTimeRaw: 13000
Done camera: False
[25 93 32]
ExposureTimeRaw: 33000
Done camera: False
[ 

Done camera: False
[63 95 10]
ExposureTimeRaw: 11000
Done camera: False
[56 57  4]
ExposureTimeRaw: 5000
Done camera: False
[31 93 24]
ExposureTimeRaw: 25000
Done camera: False
[59 83 91]
ExposureTimeRaw: 285000


In [9]:
#model.save("model_corning_basler_new_obs_03")

In [ ]:
#tensorboard --logdir=C:\\Users\\CIG\\Documents\\MATLAB